In [1]:
localaddress = "http://YOUR_IP_ADDRESS:PORT"

In [2]:
import os

# 清除代理变量
os.environ.pop('http_proxy', None)
os.environ.pop('https_proxy', None)
os.environ.pop('ftp_proxy', None)
os.environ.pop('all_proxy', None)
os.environ.pop('HTTP_PROXY', None)
os.environ.pop('HTTPS_PROXY', None)
os.environ.pop('FTP_PROXY', None)
os.environ.pop('ALL_PROXY', None)

# 验证代理变量是否已清除
print(os.environ.get('http_proxy'))
print(os.environ.get('https_proxy'))
print(os.environ.get('ftp_proxy'))
print(os.environ.get('all_proxy'))
print(os.environ.get('HTTP_PROXY'))
print(os.environ.get('HTTPS_PROXY'))
print(os.environ.get('FTP_PROXY'))
print(os.environ.get('ALL_PROXY'))

None
None
None
None
None
None
None
None


In [3]:
import selenium
import sys
import platform

print(f"Python版本: {sys.version}")
print(f"Selenium版本: {selenium.__version__}")
print(f"平台信息: {platform.platform()}")

# 检查关键模块
try:
    from selenium.webdriver.common.selenium_manager import SeleniumManager
    print("✓ Selenium Manager 模块可用")
except ImportError:
    print("✗ Selenium Manager 模块不可用")

# 检查 Edge 浏览器是否存在
import os
edge_path = r"/path/to/file"
msedge_path = r""

if os.path.exists(edge_path):
    print("✓ Edge 浏览器可执行文件存在")
else:
    print("✗ Edge 浏览器可执行文件不存在")

if os.path.exists(edge_path):
    print("✓ Edge driver 可执行文件存在")
else:
    print("✗ Edge driver 执行文件不存在")

Python版本: 3.10.18 (main, Jun  5 2025, 13:14:17) [GCC 11.2.0]
Selenium版本: 4.31.1
平台信息: Linux-6.6.87.2-microsoft-standard-WSL2-x86_64-with-glibc2.35


✓ Selenium Manager 模块可用
✓ Edge 浏览器可执行文件存在
✓ Edge driver 可执行文件存在


In [4]:
from selenium import webdriver
from selenium.webdriver.edge.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time
import requests
from bs4 import BeautifulSoup
import json

# 测试连接
try:
    response = requests.get(localaddress, timeout=5)
    print("✅ Windows EdgeDriver 服务正常")
except Exception as e:
    print(f"❌ 无法连接到 Windows 服务: {e}")

✅ Windows EdgeDriver 服务正常


In [5]:
# ==================== 无 import：第1个ipo_bigfont → 下一张t_content 前三列（含表头） ====================
CONFIG = {
    "REMOTE_URL": localaddress,
    "TARGET_URL": "https://example.com",
    "CSV_NAME"  : "numbers.csv",
    "TIMEOUT"   : 15,
    "HEADLESS"  : False
}

def start_remote_driver():
    opts = webdriver.EdgeOptions() if hasattr(webdriver, 'EdgeOptions') else Options()
    opts.set_capability("pageLoadStrategy", "eager")
    if CONFIG["HEADLESS"]:
        opts.add_argument("--headless")
    driver = webdriver.Remote(command_executor=CONFIG["REMOTE_URL"], options=opts)
    driver.set_page_load_timeout(CONFIG["TIMEOUT"])
    return driver

def extract_bigfont_tcontent3cols(driver):
    # 只等任意一个 td 出现（祖先-后代形式）
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located(
            (By.XPATH, '//div[contains(@class,"ipo_bigfont")][1]//table[@class="t_content"][1]//td')
        )
    )
    target_table = driver.find_element(By.XPATH,
        '//div[contains(@class,"ipo_bigfont")][1]//table[@class="t_content"][1]')

    # 表头前三列
    head = [th.text.strip() for th in target_table.find_elements(By.CSS_SELECTOR, "thead tr th")[:3]]
    # 数据前三列
    rows = target_table.find_elements(By.CSS_SELECTOR, "tbody tr")
    data = []
    for tr in rows:
        try:
            cells = tr.find_elements(By.CSS_SELECTOR, "td")[:3]
            if not cells: continue
            row = [cells[0].text.strip(),
                   int(cells[1].text),
                   int(cells[2].text)]
            data.append(row)
        except ValueError:
            continue
    return pd.DataFrame(data, columns=head if head else ["市场", "发行家数", "破板家数"])

def main_bigfont_tcontent3cols():
    driver = start_remote_driver()
    try:
        try:
            driver.get(CONFIG["TARGET_URL"])
        except TimeoutException:
            driver.execute_script("window.stop();")

        df_new = extract_bigfont_tcontent3cols(driver)
        df_new.to_csv(CONFIG["CSV_NAME"], mode="a", header=not os.path.exists(CONFIG["CSV_NAME"]), index=False)
        print(f"✅ 追加 {len(df_new)} 条（含表头）到 {CONFIG['CSV_NAME']}")
        return df_new
    finally:
        driver.quit()

display(main_bigfont_tcontent3cols())

✅ 追加 3 条（含表头）到 numbers.csv


,市场,发行家数,破板家数
0,上海,23,23
1,深圳,46,48
2,科创,15,13
